# Programa que calcula previsão para o dia seguinte

In [1]:
print('ola mundo')
import tensorflow as tf
print(tf.__version__)

ola mundo


2022-11-28 01:31:36.204788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-28 01:31:36.205163: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.7.0


In [2]:
import requests
import pandas as pd
import numpy as np

In [3]:
urlRedes = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaParaPrevisao"
urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisaoTreino'

In [4]:
def incluiEntrada(treino):
    urlEntrada = "http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoProximoB3?idRegra=" + str(treino['regraProjecaoId']) + "&idGrupo=" + str(treino['grupoAcaoId'])
    print(urlEntrada)
    r = requests.get(urlEntrada)
    j = r.json()
    dfEntradas = pd.DataFrame.from_dict(j)
    print(dfEntradas.head())
    return dfEntradas

In [5]:
def montaDados(rede):
    global dfDados
    global X
    urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemProximoDia?qtdeDias={}".format(rede.qtdeDias1)
    print('conexao exemplos')
    print(urlExemplos)
    r = requests.get(urlExemplos)
    j = r.json()
    dfDados = pd.DataFrame.from_dict(j)
    print(dfDados.shape)
    dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
    X = np.stack(dfTratadoX.values)
    print("X")
    print(X.shape)
    dfDados.drop('campoX', inplace=True, axis=1)

In [10]:
def trataRede(rede):
    montaDados(rede)
    print('ID:' , rede['id'])
    nome_arquivo = '/root/executa-rede/redes/treino_rede' + str(rede['id']) + '.h5'
    print(nome_arquivo)
    modelo = tf.keras.models.load_model(nome_arquivo)
    modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(rede['id'])
    dfDados['tipoCompraVenda'] = rede['tipoCompraVenda']
    dfEntradas = incluiEntrada(rede)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    json = dfPrevisao.to_dict(orient='list')
    response = requests.post(urlPrevisao, data=json)
    print(json)
    print(response)

In [11]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [12]:
print('conexao redes')
r = requests.get(urlRedes)
j = r.json()
dfRede = pd.DataFrame.from_dict(j)
print(dfRede.head())

conexao redes
    id  grupoAcaoId  periodoTreinoRedeId tipoExemploTreinoId  regraProjecaoId  \
0   43           12                    1                None               68   
1   77           12                    1                None               66   
2   79           12                    1                None               68   
3   80           12                    1                None               69   
4  276           12                    1                None               48   

   redeNeuralId  ativoPrevisao  limiteParaEntrada  qtdeLucroTeste  \
0             2              1               0.23               8   
1             1              1               0.26               9   
2             1              1               0.23               9   
3             1              1               0.30               8   
4             6              1               0.42               9   

   qtdePrejuizoTeste  ...         dataPrevisaoTeste  mediaValorTeste  \
0           

In [13]:
dfRede.apply(trataRede, axis='columns')

conexao exemplos
http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemProximoDia?qtdeDias=120
(86, 3)
X
(86, 1800)
ID: 43
/root/executa-rede/redes/treino_rede43.h5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1800)              3241800   
                                                                 
 dense_1 (Dense)             (None, 1400)              2521400   
                                                                 
 dense_2 (Dense)             (None, 1000)              1401000   
                                                                 
 dense_3 (Dense)             (None, 600)               600600    
                                                                 
 dense_4 (Dense)             (None, 200)               120200    
                                                                 
 dens

http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoProximoB3?idRegra=66&idGrupo=12
  ticker valorReferencia valorEntrada
0  ABEV3            None         None
1  AERI3            None         None
2  ALPA4            None         None
3  ALSO3            None         None
4  AMAR3            None         None
{'ticker': ['ABEV3', 'AERI3', 'ALPA4', 'ALSO3', 'AMAR3', 'ANIM3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BPAN4', 'BRAP4', 'BRFS3', 'BRKM5', 'BRML3', 'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSMG3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3', 'HYPE3', 'ITUB4', 'JBSS3', 'JHSF3', 'KLBN11', 'LIGT3', 'LREN3', 'LWSA3', 'MGLU3', 'MOVI3', 'MRFG3', 'MRVE3', 'MULT3', 'MYPK3', 'NEOE3', 'NTCO3', 'PCAR3', 'PETR4', 'POSI3', 'PRIO3', 'PTBL3', 'QUAL3', 'RADL3', 'RAIL3', 'RAPT4',

(86, 3)
X
(86, 1800)
ID: 80
/root/executa-rede/redes/treino_rede80.h5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              1801000   
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 dense_2 (Dense)             (None, 1)                 501       
                                                                 
Total params: 2,302,001
Trainable params: 2,302,001
Non-trainable params: 0
_________________________________________________________________
http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoProximoB3?idRegra=69&idGrupo=12
  ticker valorReferencia valorEntrada
0  ABEV3            None         None
1  AERI3            None         Non

http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoProximoB3?idRegra=48&idGrupo=12


KeyboardInterrupt: 